<a href="https://colab.research.google.com/github/YairYarden/DeepCascadingAMC/blob/feature%2Fload_dataset/DeepCascadingAMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code implements the paper :

"**Deep cascading network architecture for robust automatic modulation classification**"

by : 
Lintianran Weng a, Yuan He b,⇑, Jianhua Peng a, Jianchao Zheng c, Xinyu Li b

Refs :

Loading Data : [link](https://github.com/qieaaa/Singal-CNN/blob/master/train.py)


# Imports

In [5]:
import torch
import numpy as np
from datetime import datetime 
import os

import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, SubsetRandomSampler

import ssl
import matplotlib.pyplot as plt

# check if CUDA is available
is_gpu_available = torch.cuda.is_available()

if not is_gpu_available:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# DIR_PATH = '/content/drive/MyDrive/deep_learning_project'

CUDA is not available.  Training on CPU ...


# Mount Google Drive

In [6]:
from google.colab import drive
drive.mount('/content/drive/')

drive_path = '/content/drive/MyDrive/deep_learning_project'
data_path = os.path.join(drive_path, 'RadioML2016_b/RML2016.10b.dat')

print('Drive path : ' + drive_path)
print('Data path : ' + data_path)

Mounted at /content/drive/
/content/drive/MyDrive/deep_learning_project


# Data preparation

## Load dataset from drive

In [12]:
import pickle, random
with open(data_path,'rb') as xd1: 
    Xd = pickle.load(xd1,encoding='latin1')

snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])
X = []  
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):  lbl.append((mod,snr))
X = np.vstack(X)

print('Modulation Options : ', mods)
print('SNR Options : ', snrs)

## Divide to train & Test

In [13]:
# Data
np.random.seed(2016) 
n_examples = X.shape[0]
n_train = n_examples * 0.8
train_idx = np.random.choice(range(0,n_examples), size=int(n_train), replace=False)
test_idx = list(set(range(0,n_examples))-set(train_idx)) #label
X_train = X[train_idx]
X_test =  X[test_idx]

# Labels
def to_onehot(yy):
    yy1 = np.zeros([len(yy), max(yy)+1])
    yy1[np.arange(len(yy)),yy] = 1
    return yy1

trainy =list(map(lambda x: mods.index(lbl[x][0]), train_idx))
Y_train = to_onehot(trainy)
Y_test = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), test_idx)))

# Print Shapes
print ('Train dataset shape : ', X_train.shape)
print('Train Labels shape :', Y_train.shape)
print ('Test dataset shape : ', X_test.shape)
print('Test Labels shape :', Y_test.shape)

(600000, 2, 128) [2, 128]


## Testing

In [33]:
example = X_train[1,:,:]
print(example.shape)

(2, 128)
